# Treinamento com interface de alto nível

## Importação das bibliotecas

In [1]:
# http://pytorch.org/
from os.path import exists

import torch

In [16]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from PIL import UnidentifiedImageError
from torch.utils.data import Dataset, random_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

# Dataset

In [3]:
#!/bin/bash
!curl -L -o /content/archive.zip https://www.kaggle.com/api/v1/datasets/download/kumarajarshi/life-expectancy-who

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  118k  100  118k    0     0   106k      0  0:00:01  0:00:01 --:--:--  236k


In [4]:
!unzip -q -o /content/archive.zip -d /content/life-expectancy

In [5]:
!ls /content/life-expectancy

'Life Expectancy Data.csv'


In [6]:
csv_folder = "/content/life-expectancy/Life Expectancy Data.csv"

In [8]:
df = pd.read_csv(csv_folder)
df.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [9]:
df.shape

(2938, 22)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          2938 non-null   object 
 1   Year                             2938 non-null   int64  
 2   Status                           2938 non-null   object 
 3   Life expectancy                  2928 non-null   float64
 4   Adult Mortality                  2928 non-null   float64
 5   infant deaths                    2938 non-null   int64  
 6   Alcohol                          2744 non-null   float64
 7   percentage expenditure           2938 non-null   float64
 8   Hepatitis B                      2385 non-null   float64
 9   Measles                          2938 non-null   int64  
 10   BMI                             2904 non-null   float64
 11  under-five deaths                2938 non-null   int64  
 12  Polio               

In [11]:
df.isnull().sum()

,0
Country,0
Year,0
Status,0
Life expectancy,10
Adult Mortality,10
infant deaths,0
Alcohol,194
percentage expenditure,0
Hepatitis B,553
Measles,0


In [12]:
df.duplicated().sum()

0

## Removendo dados nulos

In [14]:
for c in df.columns:
  if df[c].dtype != object:
    df.loc[df[c].isnull(), c] = df[c].mean()

<ipython-input-14-834b11856745>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '30.303948264125257' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df[c].isnull(), c] = df[c].mean()
<ipython-input-14-834b11856745>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2419.5922396187884' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df[c].isnull(), c] = df[c].mean()
<ipython-input-14-834b11856745>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '42.0357385976855' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df[c].isnull(), c] = df[c].mean()


In [15]:
df.isnull().sum()

,0
Country,0
Year,0
Status,0
Life expectancy,0
Adult Mortality,0
infant deaths,0
Alcohol,0
percentage expenditure,0
Hepatitis B,0
Measles,0


In [17]:
label_encoder_country = preprocessing.LabelEncoder()
df['Country']= label_encoder_country.fit_transform(df['Country'])

label_encoder_status = preprocessing.LabelEncoder()
df['Status']= label_encoder_status.fit_transform(df['Status'])

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          2938 non-null   int64  
 1   Year                             2938 non-null   float64
 2   Status                           2938 non-null   int64  
 3   Life expectancy                  2938 non-null   float64
 4   Adult Mortality                  2938 non-null   float64
 5   infant deaths                    2938 non-null   float64
 6   Alcohol                          2938 non-null   float64
 7   percentage expenditure           2938 non-null   float64
 8   Hepatitis B                      2938 non-null   float64
 9   Measles                          2938 non-null   float64
 10   BMI                             2938 non-null   float64
 11  under-five deaths                2938 non-null   float64
 12  Polio               

In [19]:
df.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,0,2015.0,1,65.0,263.0,62.0,0.01,71.279624,65.0,1154.0,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,0,2014.0,1,59.9,271.0,64.0,0.01,73.523582,62.0,492.0,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,0,2013.0,1,59.9,268.0,66.0,0.01,73.219243,64.0,430.0,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,0,2012.0,1,59.5,272.0,69.0,0.01,78.184215,67.0,2787.0,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,0,2011.0,1,59.2,275.0,71.0,0.01,7.097109,68.0,3013.0,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [22]:
scaler = preprocessing.StandardScaler()
new_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [23]:
new_df.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,-1.691042,1.626168,0.459399,-0.444448,0.791586,0.268824,-1.172958,-0.335570,-0.705861,-0.110384,...,-3.278638,0.925806,-0.732952,-0.323445,-0.525248,0.389975,2.813130,2.773279,-0.725401,-0.579931
1,-1.691042,1.408800,0.459399,-0.980950,0.856072,0.285786,-1.172958,-0.334441,-0.838704,-0.168124,...,-1.051482,0.934140,-0.859877,-0.323445,-0.523083,-0.230936,2.881408,2.817902,-0.740050,-0.610570
2,-1.691042,1.191432,0.459399,-0.980950,0.831890,0.302749,-1.172958,-0.334594,-0.750142,-0.173531,...,-0.880163,0.913306,-0.775260,-0.323445,-0.521632,0.352715,2.926927,2.862526,-0.769349,-0.641209
3,-1.691042,0.974064,0.459399,-1.023029,0.864132,0.328193,-1.172958,-0.332096,-0.617299,0.032045,...,-0.666013,1.075815,-0.648335,-0.323445,-0.518723,-0.168315,2.972446,2.929461,-0.803531,-0.671847
4,-1.691042,0.756696,0.459399,-1.054587,0.888314,0.345155,-1.172958,-0.367862,-0.573018,0.051757,...,-0.623183,0.804966,-0.606027,-0.323445,-0.564893,-0.181666,3.040724,2.974085,-0.847480,-0.763764


In [25]:
y = new_df['Life expectancy ']

In [28]:
y.shape
y = y.values.reshape(-1, 1)
y.shape

(2938, 1)

In [30]:
x = new_df[new_df.columns.difference(['Life expectancy '])]
x.shape

(2938, 21)

In [31]:
class CustomDataset(Dataset):
  def __init__(self, x, y):
    super(CustomDataset, self).__init__()
    self.x = torch.Tensor(x)
    self.y = torch.Tensor(y)

  def __getitem__(self, index):
    return self.x[index], self.y[index]

  def __len__(self):
    return len(self.x)


In [34]:
dataset = CustomDataset(x.to_numpy(), y)

In [35]:
dataset_train, dataset_test = random_split(dataset, [0.7, 0.3])

In [36]:
len(dataset_train)

2057

In [37]:
len(dataset_test)

881

## DataLoader

In [38]:
use_cuda = torch.cuda.is_available()

torch.manual_seed(1111)

device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': 500}
test_kwargs = {'batch_size': 100}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

train_loader = torch.utils.data.DataLoader(dataset_train,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset_test, **test_kwargs)

## Criação da rede

In [39]:
input_size = 21
output_size = 1
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        x = self.fc(x)
        output = x
        return output

model = Net()

In [40]:
model

Net(
  (fc): Sequential(
    (0): Linear(in_features=21, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ReLU()
    (8): Linear(in_features=32, out_features=1, bias=True)
  )
)

## Predict

## Treinamento

In [41]:
model(dataset_train[5][0])

tensor([-0.0337], grad_fn=<ViewBackward0>)

### Criando o objeto de treinamento

In [42]:
def train(log_interval, dry_run, model, device, train_loader, optimizer, epoch, criterion):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if dry_run:
          break
    print('Train Epoch: {} \tLoss: {:.6f}'.format(
        epoch, loss.data))

In [43]:
def test(model, device, test_loader, criterion):
    model.eval()
    total_loss = 0
    total_samples = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item() * target.size(0)
            total_samples += target.size(0)

            # Calculate Mean Squared Error
        mse = total_loss / total_samples
        print(f'Mean Squared Error on validation set: {mse:.4f}')
    return mse

## Avaliação

In [47]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1)
criterion = nn.MSELoss()

epochs = 8
best_loss = test(model, device, test_loader, criterion)
for epoch in range(1, epochs + 1):
    train(10, False, model, device, train_loader, optimizer, epoch, criterion)
    loss = test(model, device, test_loader, criterion)
    if loss < best_loss:
      best_loss = loss
      torch.save(model.state_dict(), "catsdogs_nn.pt")

Mean Squared Error on validation set: 0.9946
Train Epoch: 1 	Loss: 0.769273
Mean Squared Error on validation set: 0.9483
Train Epoch: 2 	Loss: 0.733642
Mean Squared Error on validation set: 0.8916
Train Epoch: 3 	Loss: 0.831585
Mean Squared Error on validation set: 0.7425
Train Epoch: 4 	Loss: 0.637271
Mean Squared Error on validation set: 0.4065
Train Epoch: 5 	Loss: 0.245322
Mean Squared Error on validation set: 0.2458
Train Epoch: 6 	Loss: 0.262408
Mean Squared Error on validation set: 0.2916
Train Epoch: 7 	Loss: 0.182315
Mean Squared Error on validation set: 0.2914
Train Epoch: 8 	Loss: 0.156120
Mean Squared Error on validation set: 0.1889


In [ ]:
aux = torch.load("mnist_cnn.pt")

<ipython-input-19-cb91ad9ab067>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  aux = torch.load("mnist_cnn.pt")


In [ ]:
aux

OrderedDict([('layer1.weight',
              tensor([[-0.0003, -0.0271,  0.0240,  ...,  0.0353,  0.0151, -0.0153],
                      [-0.0208, -0.0123,  0.0298,  ...,  0.0074, -0.0148, -0.0048],
                      [ 0.0010, -0.0267, -0.0105,  ...,  0.0171,  0.0191,  0.0073],
                      ...,
                      [-0.0194,  0.0303,  0.0269,  ...,  0.0003,  0.0118, -0.0055],
                      [ 0.0055, -0.0117, -0.0062,  ..., -0.0226, -0.0015, -0.0182],
                      [ 0.0190, -0.0259, -0.0127,  ..., -0.0030,  0.0031, -0.0264]],
                     device='cuda:0')),
             ('layer1.bias',
              tensor([-0.0177, -0.0261, -0.0074,  ..., -0.0339, -0.0301,  0.0227],
                     device='cuda:0')),
             ('layer2.weight',
              tensor([[-0.0271, -0.0070, -0.0012,  ..., -0.0216,  0.0065, -0.0299],
                      [-0.0017,  0.0243,  0.0073,  ...,  0.0054, -0.0276,  0.0368],
                      [-0.0211, -0.0065,  0.00

In [ ]:
model.load_state_dict(aux)

<All keys matched successfully>